In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,regexp_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
import warnings
warnings.filterwarnings('ignore')
import re

In [2]:
df=pd.read_csv('train_2kmZucJ.csv')
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [3]:
df.shape

(7920, 3)

In [4]:
tweet=df['tweet']
label=df['label']

In [5]:
x_train,x_test,y_train,y_test=train_test_split(tweet,label,test_size=.20,random_state=42)

In [6]:
stop_words=stopwords.words('english')
lem=WordNetLemmatizer()

In [7]:
def preprocessing(tweet):
    token_no_url=re.sub(r'http\S+', ' ',tweet)
    final = ' '.join([re.sub(r"[^a-zA-Z0-9]+", ' ', k) for k in token_no_url.split("\n")])
    tokens=word_tokenize(final,language='english')
    norm_tokens=[lem.lemmatize(i.lower(),pos='v') for i in tokens]
    filter_token=[token.lower() for token in norm_tokens if token.lower() not in stop_words]
    return " ".join(filter_token)
    
    
    
    

In [8]:
x_train=x_train.apply(lambda x :preprocessing(x))

In [9]:
x_test=x_test.apply(lambda x:preprocessing(x))

In [10]:
x_train.head()


4252    cool car wash idea theisland bankholidaymonday...
4428    photo 35th birthday sony walkman thinkgeek nob...
7374    ipads biggest pile fuck planet want throw fuck...
1410    yearbook hmmmmm instagram instagood together f...
7896    piss macbook crash apple company nothing apple...
Name: tweet, dtype: object

In [11]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv=CountVectorizer()


In [17]:
x_train_mat=cv.fit_transform(x_train)
x_test_mat=cv.transform(x_test)

In [ ]:
cv.get_feature_names()

In [ ]:
mnb=MultinomialNB()
mnb.fit(x_train_mat.toarray(),y_train)

In [ ]:
from sklearn.metrics import f1_score,roc_auc_score,roc_curve
pred=mnb.predict(x_test_mat.toarray())
print(f1_score(y_test,pred,average='weighted'))

In [ ]:
import matplotlib.pyplot as plt
prob=mnb.predict_proba(x_test_mat)[::,1]
fpr,tpr,threshold=roc_curve(y_test,prob)
plt.xlabel('FPR')
plt.ylabel("TPR")
plt.plot(fpr,tpr)


In [ ]:
tuning=pd.DataFrame({'FPR':fpr,'TPR':tpr,"THRESHOLD":threshold})

tuning[tuning['TPR']>.80][30:40]

In [ ]:
new_pred=[]
for i in prob:
    if i >0.372833:
        new_pred.append(1)
    elif i<0.372833:
        new_pred.append(0)

In [ ]:
print(f1_score(y_test,new_pred,average='weighted'))

In [ ]:

test=pd.read_csv('test_oJQbWVk.csv')
test.head()

In [ ]:
test['cleaned_tweets']=test['tweet'].apply(lambda x:preprocessing(x))

In [ ]:

test_mat=cv.transform(test['cleaned_tweets'])

In [ ]:
test_pre=mnb.predict(test_mat.toarray())

In [ ]:
#prob1=mnb.predict_proba(test_mat)[::,1]

In [ ]:
'''test_pred1=[]
for i in prob1:
    if i >0.372833:
        test_pred1.append(1)
    elif i<0.372833:
        test_pred1.append(0)'''

In [ ]:
submisssion=pd.DataFrame({'id':test["id"],'label':test_pre})


In [ ]:
submisssion.to_csv(r'submission_tweets.csv', index = False)